In [0]:
# boilerplate

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.window import *

# create spark session
# /Workspace/Users/datakraft867@gmail.com/books.csv
spark = SparkSession.builder.appName("books_query").getOrCreate()
books_df = spark.read.csv("/Volumes/datakraft_batch1/default/datasets/books.csv", header=True, inferSchema=True)

In [0]:
# count the number of rows
books_df.count()

11127

In [0]:
books_df.show(10)

+---------+--------------------+--------------+------+----------+-------------+-------------+----------------+---------------+-------------+------------------+--------------------+
|num_pages|             authors|average_rating|bookID|      isbn|       isbn13|language_code|publication_date|      publisher|ratings_count|text_reviews_count|               title|
+---------+--------------------+--------------+------+----------+-------------+-------------+----------------+---------------+-------------+------------------+--------------------+
|      652|J.K. Rowling/Mary...|          4.57|     1|0439785960|9780439785969|          eng|       9/16/2006|Scholastic Inc.|      2095690|             27591|Harry Potter and ...|
|      870|J.K. Rowling/Mary...|          4.49|     2|0439358078|9780439358071|          eng|        9/1/2004|Scholastic Inc.|      2153167|             29221|Harry Potter and ...|
|      352|        J.K. Rowling|          4.42|     4|0439554896|9780439554893|          eng|  

In [0]:
# display top 10 rows
display(books_df.limit(10))

num_pages,authors,average_rating,bookID,isbn,isbn13,language_code,publication_date,publisher,ratings_count,text_reviews_count,title
652,J.K. Rowling/Mary GrandPré,4.57,1,0439785960,9780439785969,eng,9/16/2006,Scholastic Inc.,2095690,27591,Harry Potter and the Half-Blood Prince (Harry Potter #6)
870,J.K. Rowling/Mary GrandPré,4.49,2,0439358078,9780439358071,eng,9/1/2004,Scholastic Inc.,2153167,29221,Harry Potter and the Order of the Phoenix (Harry Potter #5)
352,J.K. Rowling,4.42,4,0439554896,9780439554893,eng,11/1/2003,Scholastic,6333,244,Harry Potter and the Chamber of Secrets (Harry Potter #2)
435,J.K. Rowling/Mary GrandPré,4.56,5,043965548X,9780439655484,eng,5/1/2004,Scholastic Inc.,2339585,36325,Harry Potter and the Prisoner of Azkaban (Harry Potter #3)
2690,J.K. Rowling/Mary GrandPré,4.78,8,0439682584,9780439682589,eng,9/13/2004,Scholastic,41428,164,Harry Potter Boxed Set Books 1-5 (Harry Potter #1-5)
152,W. Frederick Zimmerman,3.74,9,0976540606,9780976540601,en-US,4/26/2005,Nimble Books,19,1,Unauthorized Harry Potter Book Seven News: Half-Blood Prince Analysis and Speculation
3342,J.K. Rowling,4.73,10,0439827604,9780439827607,eng,9/12/2005,Scholastic,28242,808,Harry Potter Collection (Harry Potter #1-6)
815,Douglas Adams,4.38,12,0517226952,9780517226957,eng,11/1/2005,Gramercy Books,3628,254,The Ultimate Hitchhiker's Guide: Five Complete Novels and One Story (Hitchhiker's Guide to the Galaxy #1-5)
815,Douglas Adams,4.38,13,0345453743,9780345453747,eng,4/30/2002,Del Rey Books,249558,4080,The Ultimate Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy #1-5)
215,Douglas Adams,4.22,14,1400052920,9781400052929,eng,8/3/2004,Crown,4930,460,The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy #1)


In [0]:
# create temp value
books_df.createOrReplaceTempView("books_tbl")

In [0]:
# How many total books are present in the database?
books_cnt = spark.sql(\
    f"""
    SELECT COUNT(isbn) as books_cnt FROM books_tbl
    """).show()

+---------+
|books_cnt|
+---------+
|    11127|
+---------+



In [0]:
# what is the total page count of all books published by Bill Bryson?
# verify
books_df.printSchema()

root
 |-- num_pages: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- bookID: integer (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- publication_date: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- ratings_count: integer (nullable = true)
 |-- text_reviews_count: integer (nullable = true)
 |-- title: string (nullable = true)



In [0]:
# cast num_pages to integer
books_df = books_df.withColumn("num_pages", col("num_pages").cast("integer"))

In [0]:
# sql
tot_pg_bb = spark.sql(\
    f"""
    SELECT SUM(num_pages)
    FROM books_tbl
    WHERE authors = "Bill Bryson"
    """).show()


+--------------+
|sum(num_pages)|
+--------------+
|        6865.0|
+--------------+



In [0]:
# for books published in 1943, what is the average book rating?
# get overall rating for that year 
# extract year from publication date

avg_rating = spark.sql(\
  f"""
  WITH CTE AS (
  SELECT 
  CAST(RIGHT(publication_date, 4) AS INT) AS yr,
  ROUND(AVG(average_rating),2) AS avg_rating
  FROM books_tbl
  GROUP BY 1
  ORDER BY 1
  )

  SELECT yr, avg_rating FROM CTE WHERE yr = 1943;
  """).show()

+----+----------+
|  yr|avg_rating|
+----+----------+
|1943|      4.21|
+----+----------+



In [0]:
books_df.printSchema()

root
 |-- num_pages: integer (nullable = true)
 |-- authors: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- bookID: integer (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- publication_date: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- ratings_count: integer (nullable = true)
 |-- text_reviews_count: integer (nullable = true)
 |-- title: string (nullable = true)



In [0]:
# How many books did each author release every year? Please note that the authors field may contain multiple authors. Output should include author, published_year_cnt

release_auth = spark.sql(\
  f"""
  WITH CTE AS (
  SELECT 
  CAST(RIGHT(publication_date, 4) AS INT) AS yr, title, authors
  FROM books_tbl
  group by 1,2,3
  order by 1
  )

  SELECT * FROM CTE;
  """).display()

yr,title,authors
0,Brown's Star Atlas: Showing All The Bright Stars With Full Instructions How To Find And Use Them For Navigational Purposes And Department Of Trade Examinations.,Brown
4,The Tolkien Fan's Medieval Reader,David E. Smith (Turgon of TheOneRing.net
4,Patriots (The Coming Collapse),James Wesley
6,Streetcar Suburbs: The Process of Growth in Boston 1870-1900,Sam Bass Warner
1900,Consider the Lilies,Iain Crichton Smith/Isobel Murray
1913,On Duties (De Officiis),Marcus Tullius Cicero/Walter Miller
1914,Agricola / Germania / Dialogue on Oratory,Tacitus/Maurice Hutton/Robert Maxwell Ogilvie/E.H. Warmington/William Peterson/Michael Winterbottom
1919,History of the Peloponnesian War: Bk. 1-2,Thucydides/C.F. Smith
1921,History of the Peloponnesian War: Bk. 5-6,Thucydides/C.F. Smith
1921,The Library 1 Books 1-3.9,Apollodorus/James George Frazer


In [0]:
# How many books did each author release every year? Please note that the authors field may contain multiple authors. Output should include author, published_year_cnt

release_auth = spark.sql(\
  f"""
  WITH CTE AS (
  SELECT 
  CAST(RIGHT(publication_date, 4) AS INT) AS yr, title, authors
  FROM books_tbl
  group by 1, 2, 3
  order by 1
  )

  SELECT 
  distinct authors, yr,
  COUNT(title) over (partition by authors, yr order by yr) AS published_year_cnt 
  FROM CTE;
  """).display()

authors,yr,published_year_cnt
A.B. Yehoshua/Hillel Halkin,2004,1
A.B. Yehoshua/Hillel Halkin,2006,1
A.D.P. Briggs/Leo Tolstoy/Fyodor Dostoyevsky,1994,1
A.E. Cunningham/Harlan Ellison/Charles F. Miller/Terry Dowling/Paul Rhoads/Jack Vance/Tom Shippey/Gene Wolfe/David Langford/Dan Simmons/David Mathew,2000,1
A.J. Jacobs,2005,1
A.M. Homes,2007,1
A.N. Roquelaure/Anne Rice,1984,1
A.N. Roquelaure/Anne Rice,1999,3
A.S. Byatt,1991,1
A.S. Byatt,1994,1


In [0]:
# display publishers like Oxford University Press

publisher = spark.sql(\
    f"""
    SELECT DISTINCT publisher 
    FROM books_tbl
    WHERE publisher like '%Oxford University Press%'
    """).display()

publisher
Oxford University Press
Oxford University Press USA


In [0]:
# Oxford University Press and published in 1998, list the books? ;

title = spark.sql(\
f"""
WITH CTE AS (
SELECT publisher, title,
CAST(RIGHT(publication_date, 4) AS INT) AS yr
FROM books_tbl
)
select *
from CTE
where yr = 1998 and publisher like '%Oxford University Press%' ;
""").display()

publisher,title,yr
Oxford University Press,War and Peace,1998
Oxford University Press,The Histories,1998
Oxford University Press,Antigone; Oedipus the Kingn; Electra,1998
Oxford University Press USA,Moby Dick: or The White Whale (Oxford Illustrated Classics),1998
Oxford University Press,The Bostonians,1998
Oxford University Press,The Canterbury Tales,1998
Oxford University Press,Waverley,1998
Oxford University Press,The Karamazov Brothers,1998
Oxford University Press,The Castle of Otranto,1998
Oxford University Press USA,Faust Part One,1998


In [0]:
# Oxford University Press and published in 1988, list the books? ;
title = spark.sql(\
f"""
WITH CTE AS (
SELECT publisher, title,
CAST(RIGHT(publication_date, 4) AS INT) AS yr
FROM books_tbl
)
select 
case
when yr = 1988 and publisher like '%Oxford University Press%' then publisher || ' - ' || title
else 'no books published in 1988 by Oxford University Press'
end as book_info
from CTE
limit 1
""").display()

book_info
no books published in 1988 by Oxford University Press


In [0]:
# all books published by Dan Brown (even as co-author) # author, book_title, publication_date, 

publisher = spark.sql(\
    f"""
    SELECT authors, title, publication_date
    FROM books_tbl
    WHERE authors like '%Dan Brown%'
    group by authors, title, publication_date
    order by authors, title, publication_date desc
    """).display()


authors,title,publication_date
Dan Brown,Angels & Demons (Robert Langdon #1),7/28/2003
Dan Brown,Angels & Demons (Robert Langdon #1),5/3/2005
Dan Brown,Angels & Demons (Robert Langdon #1),4/1/2006
Dan Brown,Deception Point,8/1/2005
Dan Brown,Deception Point,5/1/2004
Dan Brown,Deception Point,4/1/2006
Dan Brown,The Da Vinci Code,3/28/2006
Dan Brown,The Da Vinci Code (Robert Langdon #2),3/28/2006
Dan Brown,The Da Vinci Code (Robert Langdon #2),3/18/2003
Dan Brown,Ángeles y demonios (Robert Langdon #1),12/1/2005


In [0]:
# How many books did Dan Brown publish? # 

tot_books = spark.sql(\
    f"""
    SELECT authors, count(title) as total_books
    FROM books_tbl
    WHERE authors like '%Dan Brown%'
    group by authors
    order by authors
    """).display()

authors,total_books
Dan Brown,12
Dan Brown/Daniel Roche,1
Dan Brown/Eduardo García Murillo,1
Dan Brown/Juanjo Estrella,1
Dan Brown/Paul Michael,1
Dan Brown/Raúl Amundaray,1


In [0]:
# Which publisher had the books with the highest rating; 
# publisher, book_title, rating

high_rating = spark.sql(\
    f"""
    SELECT publisher, title, average_rating
    FROM books_tbl
    where average_rating = (
        select max(average_rating)
        from books_tbl)
    """).display()


publisher,title,average_rating
Oxford University Press USA,Comoediae 1: Acharenses/Equites/Nubes/Vespae/Pax/Aves,5.00
Schirmer Mosel,Willem de Kooning: Late Paintings,5.00
Teaching Resources,Literature Circle Guide: Bridge to Terabithia: Everything You Need For Successful Literature Circles That Get Kids Thinking Talking Writing—and Loving Literature,5.00
Arcadia Publishing,Middlesex Borough (Images of America: New Jersey),5.00
BradyGames,Zone of the Enders: The 2nd Runner Official Strategy Guide,5.00
Square One Publishers,The Diamond Color Meditation: Color Pathway to the Soul,5.00
Vantage Press,Bulgakov's the Master and Margarita: The Text as a Cipher,5.00
Boosey & Hawkes Inc,The Complete Theory Fun Factory: Music Theory Puzzles and Games for the Early Grades,5.00
BBC Physical Audio,The Goon Show Volume 4: My Knees Have Fallen Off!,5.00
BBC Physical Audio,The Goon Show Volume 11: He's Fallen in the Water!,5.00


In [0]:
# which book had highest text_reviews_count? # book_tile, text_reviews_count

text_review = spark.sql(\
f"""
 SELECT title, text_reviews_count
 FROM books_tbl
    WHERE text_reviews_count = (
        SELECT MAX(text_reviews_count)
        FROM books_tbl
    )
""").display()

title,text_reviews_count
Twilight (Twilight #1),94265
